In [3]:
import openai
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Set up OpenAI API key directly
openai.api_key = ""

# Initialize chat history
messages = [
    {
        "role": "assistant",
        "content": "Ask me a question about Streamlit's open-source Python library!",
    }
]

# Function to load data and set up the LLM with LlamaIndex
def load_data():
    # Read all documents in the specified directory (./data) and load them
    reader = SimpleDirectoryReader(input_dir="./data", recursive=True)
    docs = reader.load_data()
    
    # Configure the LLM model with context specifically for the Streamlit library
    llm = OpenAI(
        model="gpt-3.5-turbo",
        temperature=0.2,
        system_prompt="""You are an expert on 
        the OTL_edit dataset and your 
        job is to answer questions about theses books' information. 
        Assume that all questions are related 
        to the OTL_edit.cvs and 
        answer based only on the provided documents.""",
    )
    
    # Create an index based on the documents loaded from the specified directory
    index = VectorStoreIndex.from_documents(docs, llm=llm)
    return index

# Load data and create a chat engine for interactions based on the indexed data
index = load_data()
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True, streaming=True)

# Start chat interaction loop
print("Chat with the Streamlit docs, powered by LlamaIndex 💬🦙")
print("Ask me a question about Streamlit's open-source Python library! Type 'exit' to quit.")

while True:
    # Get user input
    prompt = input("You: ")
    if prompt.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break

    # Append user question to messages
    messages.append({"role": "user", "content": prompt})

    # Generate a response based on the indexed documents only
    response_stream = chat_engine.stream_chat(prompt)
    assistant_response = "".join(response_stream.response_gen)  # Collect streamed response
    
    # Display the assistant's response and save it in chat history
    print("Assistant:", assistant_response)
    messages.append({"role": "assistant", "content": assistant_response})


ModuleNotFoundError: No module named 'llama_index.query_engine'